### Crawl Data Berita Di WEB KOMPAS.com

In [5]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd

# Definisi fungsi untuk mendapatkan artikel dari URL
def get_articles_for_date_and_category(date_str, category, articles, max_articles):
    base_url = f'https://indeks.kompas.com/?site={category}&date={date_str}'
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Cari jumlah halaman
    paginations_last = soup.find("a", class_='paging__link paging__link--last')
    if paginations_last:
        pagination_page = int(paginations_last.get('data-ci-pagination-page', 1)) + 1
    else:
        paginations_next = soup.find("a", class_='paging__link paging__link--next')
        pagination_page = int(paginations_next.get('data-ci-pagination-page', 1)) + 1 if paginations_next else 2

    for page in range(1, pagination_page):
        allpage = f'{base_url}&page={page}'
        response = requests.get(allpage)
        soup = BeautifulSoup(response.content, "html.parser")
        articles_div = soup.find_all("div", class_="articleItem")

        for article in articles_div:
            if len(articles) >= max_articles:
                return

            img = article.find('div', class_='articleItem-img').find('img')['src'] if article.find('div', class_='articleItem-img') else None
            judul = article.find('h2', class_='articleTitle').text.strip() if article.find('h2', class_='articleTitle') else None
            url = article.find('a', class_='article-link')['href'] if article.find('a', class_='article-link') else None
            kelas = article.find('div', class_='articlePost-subtitle').text.strip() if article.find('div', class_='articlePost-subtitle') else category

            # Mengambil data konten di halaman detail berita
            if url:
                cPage = requests.get(url)
                cSoup = BeautifulSoup(cPage.content, "html.parser")

                tanggal_raw = cSoup.find('div', class_='read__time')
                tanggal = tanggal_raw.text.strip().replace(',', '').removeprefix('Kompas.com - ').removesuffix(' WIB') if tanggal_raw else date_str
                content = cSoup.find('div', class_='read__content').text.strip() if cSoup.find('div', class_='read__content') else None

                articles.append({
                    'judul': judul,
                    'tanggal': tanggal,
                    'url': url,
                    'img': img,
                    'content': content,
                    'kelas': kelas,
                    'category': category
                })

# Data kelas yang akan dicari
categories = ["EDUKASI", "HEALTH", "SPORTS", "TRAVEL", "FOOD", "OTOMOTIF", "NEWS"]
kompas_crawl = []
max_articles = 500

# Rentang tanggal untuk pengumpulan berita
start_date = '2024-03-01'
end_date = '2024-06-22'
start_date_obj = dt.datetime.strptime(start_date, "%Y-%m-%d")
end_date_obj = dt.datetime.strptime(end_date, "%Y-%m-%d")

# Loop melalui setiap tanggal dalam rentang yang diberikan
current_date_obj = start_date_obj
while current_date_obj <= end_date_obj and len(kompas_crawl) < max_articles:
    current_date_str = current_date_obj.strftime("%Y-%m-%d")

    for category in categories:
        if len(kompas_crawl) >= max_articles:
            break
        get_articles_for_date_and_category(current_date_str, category, kompas_crawl, max_articles)

    current_date_obj += dt.timedelta(days=1)

# Tampilkan hasil
print(f'Total articles collected: {len(kompas_crawl)}')


Total articles collected: 500


In [6]:
# menjadikan data frame
kmps_df = pd.DataFrame(kompas_crawl)
# liat data frame
kmps_df

,judul,tanggal,url,img,content,kelas,category
0,President University Resmi Angkat Prof. Satyan...,01/03/2024 21:04,https://www.kompas.com/edu/read/2024/03/01/210...,https://asset.kompas.com/crops/T7wZ1OgNBXdEY_D...,KOMPAS.com - President University secara resmi...,Edu,EDUKASI
1,"Cara Cek dan Link Pengumuman SNBP 2024, Diumum...",01/03/2024 20:37,https://www.kompas.com/edu/read/2024/03/01/203...,https://asset.kompas.com/crops/83mzfKOIwQ5iceA...,KOMPAS.com - Pendaftaran Seleksi Nasional Berd...,Edu,EDUKASI
2,Kemendikbud Luncurkan Program Percepatan Pemen...,01/03/2024 18:16,https://www.kompas.com/edu/read/2024/03/01/181...,https://asset.kompas.com/crops/IWhZ6NOeNc-ehTk...,KOMPAS.com - Balai Pembiayaan Pendidikan Tingg...,Edu,EDUKASI
3,Bantuan Rp 140 Juta bagi Komunitas Sastra dari...,01/03/2024 17:50,https://www.kompas.com/edu/read/2024/03/01/175...,https://asset.kompas.com/crops/t4QtF_F_Bzd9zCm...,"KOMPAS.com - Kementerian Pendidikan, Kebudayaa...",Edu,EDUKASI
4,4 Hal Penyebab Gagal Seleksi Administrasi Beas...,01/03/2024 17:22,https://www.kompas.com/edu/read/2024/03/01/172...,https://asset.kompas.com/crops/AxHwaNxH2JykVIg...,KOMPAS.com - Hasil seleksi administrasi Beasis...,Edu,EDUKASI
...,...,...,...,...,...,...,...
495,"5 Alasan Siswa Membolos Sekolah, Hasil Riset D...",03/03/2024 15:02,https://www.kompas.com/edu/read/2024/03/03/150...,https://asset.kompas.com/crops/rlFl5HmnTFBzBeF...,"KOMPAS.com - Ada banyak faktor, mengapa siswa ...",Edu,EDUKASI
496,10 Jurusan Sedikit Peminat Universitas Brawija...,03/03/2024 14:12,https://www.kompas.com/edu/read/2024/03/03/141...,https://asset.kompas.com/crops/sfbqODmc4M3ibUT...,KOMPAS.com — Pada Seleksi Nasional Berdasarkan...,Edu,EDUKASI
497,UIN Sunan Kalijaga Yogyakarta Buka Pendaftaran...,03/03/2024 13:22,https://www.kompas.com/edu/read/2024/03/03/132...,https://asset.kompas.com/crops/s_VAYjhdGRoa4-r...,KOMPAS.com - Universitas Islam Negeri Negeri S...,Edu,EDUKASI
498,"Jokowi: Jangan Sampai Kasus ""Bullying"" Ditutup...",03/03/2024 12:25,https://www.kompas.com/edu/read/2024/03/03/122...,https://asset.kompas.com/crops/yxBb9NgEq-J3kFP...,KOMPAS.com - Presiden Joko Widodo (Jokowi) mem...,Edu,EDUKASI


In [7]:
# save data crawl menjadi CSV:
kmps_df.to_csv(f'kompasCrawl.csv', index=False, encoding='utf-8', sep=';')

In [8]:
# save data crawl menjadi XLSX:
kmps_df.to_excel(f'kompasCrawl.xlsx', index=False)

In [9]:
# python to json
import json

# Simpan data ke dalam file JSON
with open(f'kompas_crawl_{start_date.replace("/", "-")}_to_{end_date.replace("/", "-")}.json', 'w', encoding='utf-8') as file:
    json.dump(kompas_crawl, file, ensure_ascii=False, indent=4)

### Crawl Data Berita Di WEB Liputan6.com

In [14]:
import requests
from bs4 import BeautifulSoup
import datetime as dt

# Definisi fungsi untuk mendapatkan artikel dari URL di Liputan6
def get_articles_for_date_and_category(date_str, category, articles, max_articles, pagin):
    base_url = f'https://www.liputan6.com/{category}/indeks/{date_str}'

    for page in range(1, pagin + 1):
        if len(articles) >= max_articles:
            return

        allpage = f'{base_url}?page={page}'
        response = requests.get(allpage)
        soup = BeautifulSoup(response.content, "html.parser")

        # Mengambil semua elemen <article> yang memiliki atribut 'data-type' dengan nilai 'Article'
        articles_div = soup.find_all(lambda tag: tag.name == 'article' and tag.get('data-type') == 'Article')

        for article in articles_div:
            if len(articles) >= max_articles:
                return

            img = article.find('img')['src'] if article.find('img') else None
            judul = article.find('h4', class_='articles--rows--item__title').text.strip() if article.find('h4', class_='articles--rows--item__title') else None
            url = article.find('a', class_='articles--rows--item__title-link')['href'] if article.find('a', class_='articles--rows--item__title-link') else None
            kelas = article.find('header', class_='articles--rows--item__header').find('a').text.strip() if article.find('header', class_='articles--rows--item__header') and article.find('header', class_='articles--rows--item__header').find('a') else category

            # Mengambil data konten di halaman detail berita
            if url:
                cPage = requests.get(url)
                cSoup = BeautifulSoup(cPage.content, "html.parser")

                tanggal_raw = cSoup.find('time', class_='read-page--header--author__datetime')
                tanggal = tanggal_raw.text.strip().replace(',', '').removeprefix('Diterbitkan ').removeprefix('Diperbarui ').removesuffix(' WIB') if tanggal_raw else date_str
                content = cSoup.find('div', class_='article-content-body__item-content').text.strip() if cSoup.find('div', class_='article-content-body__item-content') else None

                articles.append({
                    'judul': judul,
                    'tanggal': tanggal,
                    'url': url,
                    'img': img,
                    'content': content,
                    'kelas': kelas,
                    'category': category  # Tambahkan label 'category' yang sesuai dengan kategori saat ini
                })

# Data kategori yang akan dicari di Liputan6
categories = ["bisnis", "health", "lifestyle", "bola", "global", "otomotif", "tekno"]
liputan6_crawl = []
max_articles = 500
pagin = 3  # Mengambil hingga 3 halaman untuk setiap kategori

# Rentang tanggal untuk pengumpulan berita
start_date = '2024-01-01'
end_date = '2024-06-23'
start_date_obj = dt.datetime.strptime(start_date, "%Y-%m-%d")
end_date_obj = dt.datetime.strptime(end_date, "%Y-%m-%d")

# Loop melalui setiap tanggal dalam rentang yang diberikan
current_date_obj = start_date_obj
while current_date_obj <= end_date_obj and len(liputan6_crawl) < max_articles:
    current_date_str = current_date_obj.strftime("%Y/%m/%d")

    for category in categories:
        if len(liputan6_crawl) >= max_articles:
            break
        get_articles_for_date_and_category(current_date_str, category, liputan6_crawl, max_articles, pagin)

    current_date_obj += dt.timedelta(days=1)

# Tampilkan hasil
print(f'Total articles collected: {len(liputan6_crawl)}')
print(liputan6_crawl[:5])  # Menampilkan 5 artikel pertama untuk verifikasi


Total articles collected: 500
[{'judul': 'Bill Gates Harap Pelajari Kemampuan Ini dari Warren Buffett Lebih Awal, Apa Itu?', 'tanggal': '25 Mei 2024 21:08', 'url': 'https://www.liputan6.com/bisnis/read/5604247/bill-gates-harap-pelajari-kemampuan-ini-dari-warren-buffett-lebih-awal-apa-itu', 'img': 'https://cdn0-production-images-kly.akamaized.net/mjO7i61A8gNNpYunFPGitSVr9R0=/200x112/smart/filters:quality(75):strip_icc():format(webp)/kly-media-production/medias/1075017/original/015388800_1449200552-20151204-Bill-Gates-AFP-3.jpg', 'content': 'Liputan6.com, Jakarta - Miliarder Bill Gates dan Warren Buffett telah saling memberikan beberapa tips selama persahabatan mereka yang telah berlangsung selama lebih dari tiga dekade.\nMengutip CNBC, Sabtu (25/5/2024), salah satu pelajaran itu adalah sesuatu yang Bill Gates harap dapat dipelajarinya lebih awal untuk merapihkan jadwal sibuknya. Bahkan mungkin membuat Gates lebih bahagia dan produktif.\n\n BACA JUGA: Jemaah Haji 2024 Dapat Vaksin Polio 

In [16]:
# menjadikan data frame
lptn6_df = pd.DataFrame(liputan6_crawl)
# liat data frame
lptn6_df

,judul,tanggal,url,img,content,kelas,category
0,Bill Gates Harap Pelajari Kemampuan Ini dari W...,25 Mei 2024 21:08,https://www.liputan6.com/bisnis/read/5604247/b...,https://cdn0-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Miliarder Bill Gates d...",Ekonomi,bisnis
1,"Kunjungi Sekolah Kedinasan Kemenhub, Budi Kary...",25 Mei 2024 20:38,https://www.liputan6.com/bisnis/read/5604227/k...,https://cdn0-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Menteri Perhubungan (M...",Ekonomi,bisnis
2,"Tarik Investor, Skema Penjaminan Kredit UMKM P...",25 Mei 2024 20:16,https://www.liputan6.com/bisnis/read/5604213/t...,https://cdn0-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Skema penjaminan kredi...",Ekonomi,bisnis
3,"Libur Panjang Waisak 2024, Penumpang Kereta Ce...",25 Mei 2024 19:14,https://www.liputan6.com/bisnis/read/5604188/l...,https://cdn1-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Jumlah penumpang keret...",Ekonomi,bisnis
4,Pertamina Masih Hitung Kerugian Akibat Kebakar...,25 Mei 2024 18:35,https://www.liputan6.com/bisnis/read/5604170/p...,https://cdn0-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - PT Kilang Pertamina In...",Energi & Tambang,bisnis
...,...,...,...,...,...,...,...
495,8 Fakta Menarik Juno Wahana Antariksa NASA di ...,28 Mei 2024 05:00,https://www.liputan6.com/global/read/5605072/8...,https://cdn1-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Badan Penerbangan dan ...",Sains,global
496,"7 Fakta Menarik Pulsar, Si Mercusuar Kosmik",28 Mei 2024 03:00,https://www.liputan6.com/global/read/5605000/7...,https://cdn0-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Pulsar bagaikan mercus...",Sains,global
497,Melihat 6 Bintang Tertua di Alam Semesta,28 Mei 2024 01:00,https://www.liputan6.com/global/read/5604993/m...,https://cdn1-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Bintang adalah benda l...",Sains,global
498,Perusahaan Logistik Berbasis Teknologi Digital...,28 Mei 2024 20:01,https://www.liputan6.com/otomotif/read/5606426...,https://cdn1-production-images-kly.akamaized.n...,"Liputan6.com, Jakarta - Perusahaan logistik be...",Oto News,otomotif


In [17]:
# save data crawl menjadi CSV:
lptn6_df.to_csv(f'Liputan6Crawl.csv', index=False, encoding='utf-8', sep=';')

In [18]:
# save data crawl menjadi XLSX:
lptn6_df.to_excel(f'Liputan6Crawl.xlsx', index=False)

In [19]:
# python to json
import json

# Simpan data ke dalam file JSON
with open(f'liputan6_crawl_{start_date.replace("/", "-")}_to_{end_date.replace("/", "-")}.json', 'w', encoding='utf-8') as file:
    json.dump(liputan6_crawl, file, ensure_ascii=False, indent=4)